#### Import libraries

In [1]:
import librosa
import numpy as np
import os
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import time

#### Normalized Energy

In [2]:
def compute_normalized_energy(frames):
    """Tính năng lượng chuẩn hóa của từng khung."""
    energy = np.sum(frames**2, axis=1)
    if np.max(energy) > 0:
        normalized_energy = energy / np.max(energy)  # Chuẩn hóa nếu max > 0
    else:
        normalized_energy = energy  # Giữ nguyên nếu max = 0

    return normalized_energy

#### Caculated F0 Using HPS

In [99]:
def autocorrelation(signal):
    result = np.correlate(signal, signal, mode='full')
    mid = len(result) // 2
    return result[mid:]

def find_f0(signal, sr, min_freq=50, max_freq=700):
    ac = autocorrelation(signal)
    # Xác định chỉ số của độ trễ (delay) tương ứng với tần số cơ bản
    ac = ac[int(sr / max_freq):]
    # Tìm chỉ số của độ trễ tối đa
    peak = np.argmax(ac)
    # Tính F0 từ chỉ số độ trễ
    f0 = sr / (peak + int(sr / max_freq))
    
    # Nếu F0 nằm ngoài khoảng cho phép, trả về NaN
    if f0 < min_freq or f0 > max_freq:
        return 0
    
    return f0

#### Detect Baby Cry

In [120]:
def detect_audio_class(file_path, frame_length_ms=30, frame_step_ms=15, sr=16000, 
                       energy_threshold=0.005, f0_threshold=500):

    signal, sr = librosa.load(file_path, sr=sr)

    frame_length = int(frame_length_ms * sr / 1000)
    frame_step = int(frame_step_ms * sr / 1000)
    frames = librosa.util.frame(signal, frame_length=frame_length, hop_length=frame_step).T
    
    normalized_energy = compute_normalized_energy(frames)

    silence_count = 0
    voice_count = 0
    cry_count = 0

    for frame, energy in zip(frames, normalized_energy):
        if energy < energy_threshold:
            silence_count += 1
        else:
            f0 = find_f0(frame, sr)
            print(f0) 
            if f0 > f0_threshold:
                cry_count += 1
            elif f0 == 0:
                continue
            else:
                voice_count += 1

    print(f"Frames classified as Silence: {silence_count}")
    print(f"Frames classified as Voice: {voice_count}")
    print(f"Frames classified as Cry: {cry_count}")

    if silence_count > max(voice_count, cry_count):
        if voice_count > cry_count:
            return "Adult Voice"
        else:
            return "Baby Cry"
    elif cry_count > voice_count:
        return "Baby Cry"
    else:
        return "Adult Voice"

In [ ]:
if __name__ == "__main__":
    file_path = "pbl6_01_audio.wav"
    
    classification = detect_audio_class(file_path)
    print(f"Audio is classified as: {classification}")

In [6]:
def evaluate_audio_classification(dataset_path, frame_length_ms=30, frame_step_ms=15, 
                                  sr=16000, energy_threshold=0.02, f0_threshold=300):
    true_labels = []
    predicted_labels = []
    
    class_map = {'adult voice': 'Adult Voice', 'baby cry': 'Baby Cry'}

    # Tổng thời gian thực thi cho tất cả các tệp
    total_time = 0
    total_files = 0
    file_times = []  # Dùng để lưu trữ thời gian xử lý từng tệp
    
    start_time = time.time()  # Bắt đầu tính thời gian toàn bộ quá trình

    for class_name, expected_label in class_map.items():
        class_folder = os.path.join(dataset_path, class_name)
        if not os.path.isdir(class_folder):
            print(f"Directory not found: {class_folder}")
            continue
        
        print(f"Processing class: {class_name}")
        
        for file_name in os.listdir(class_folder):
            file_path = os.path.join(class_folder, file_name)
            
            # Bắt đầu đo thời gian cho từng tệp
            file_start_time = time.time()

            predicted_label = detect_audio_class(
                file_path, 
                frame_length_ms=frame_length_ms, 
                frame_step_ms=frame_step_ms, 
                sr=sr, 
                energy_threshold=energy_threshold, 
                f0_threshold=f0_threshold
            )
            
            # Ghi nhãn thực và nhãn dự đoán
            true_labels.append(expected_label)
            predicted_labels.append(predicted_label)

            # Tính thời gian cho file hiện tại
            file_end_time = time.time()
            file_time = file_end_time - file_start_time
            total_time += file_time
            total_files += 1

            # Lưu thời gian vào danh sách
            file_times.append(file_time)

    # Kết thúc đo thời gian
    end_time = time.time()
    elapsed_time = end_time - start_time

    # Tính độ chính xác
    accuracy = accuracy_score(true_labels, predicted_labels) * 100
    print("Độ chính xác: {:.2f}%".format(accuracy))

    # Tính và in độ chính xác theo từng lớp
    print("\nĐộ chính xác mỗi lớp:")
    for label in class_map.values():
        class_true = [1 if l == label else 0 for l in true_labels]
        class_pred = [1 if l == label else 0 for l in predicted_labels]
        class_accuracy = accuracy_score(class_true, class_pred) * 100
        print(f"- {label}: {class_accuracy:.2f}%")

    # Báo cáo chi tiết
    # print("\nClassification Report:")
    print(classification_report(true_labels, predicted_labels, target_names=class_map.values(), zero_division=0))

    # # In tổng thời gian cho tất cả quá trình đánh giá
    print(f"\nTổng thời gian xử lý: {elapsed_time:.4f} seconds")

    # # Tính thời gian trung bình xử lý mỗi tệp
    if total_files > 0:
        avg_time_per_file = total_time / total_files
        std_dev_time_per_file = np.std(file_times)  # Tính độ lệch chuẩn từ danh sách file_times
        print(f"Trung bình (Mean) của 1 tệp: {avg_time_per_file:.4f} seconds")
        print(f"Độ lệch chuẩn (Std Dev) của 1 tệp: {std_dev_time_per_file:.4f} seconds")
        
    # Ma trận nhầm lẫn
    cm = confusion_matrix(true_labels, predicted_labels, labels=list(class_map.values()))
    print("\nMa trận nhầm lẫn:")
    print(cm)

In [ ]:
if __name__ == '__main__':
    dataset_testing_path = 'datasets_testing'
    evaluate_audio_classification(dataset_testing_path)